# Что такое NLP?

**NLP** — это область машинного обучения, которая помогает компьютеру понимать и обрабатывать текст и речь на человеческом языке.

Основная задача — преобразовать текст в числовой формат, чтобы его можно было изучать стандартными ML‑моделями.

## Классификация текста

Задачи классификации текста по категориям, отличия спам/не спам, положительная/отрицательная рецензия и т.п.

Общий процесс решения задачи:
1. Сбор/загрузка данных
2. Предобработка: очистка, токенизация, удаление стоп-слов
3. Векторизация текста
4. Обучение классификатора
5. Оценка качества модели
6. Настройка гиперпараметров и дообучение

## Feature extraction: BoW и TF‑IDF

### Bag‑of‑Words (BoW)

**BoW** представляет текст как мешок слов — порядок слов игнорируется, важен только счёт каждой позиции.

Для корпуса документов строится словарь всех уникальных слов, и каждому документу соответствует вектор: количество каждого слова в нём.

Пример: два документа
```
Текст A: "John likes movies movies"
Текст B: "Mary likes football games"
```
BoW-представление:
```
Doc A: {"John":1, "likes":1, "movies":2, …}
Doc B: {..., "football":1, "games":1}
```

Плюсы: простота, легко реализовать, быстро и эффективно для небольших задач.

Минусы: игнорирует порядок слов, сам факт часто встречаются стоп-слова ("and", "the") — они занимают много места, но не несут смысла.

### TF‑IDF (Term Frequency–Inverse Document Frequency)

Усиливает BoW, добавляя важность слова относительно всего корпуса документов.

TF (Term Frequency) — частота слова в отдельном документе, часто нормируется:

$\text{tf}(t,d) = \frac{\text{вхождений терма }t}{\text{общая длина документа}}$

IDF (Inverse Document Frequency) — мера редкости терма в корпусе:

$\text{idf}(t,d)=log{\frac{N}{n_t}}$

где $𝑁$ - число документов, $n_t$ - сколько документов содержат $t$. Чем реже слово, тем выше IDF.

Итоговый вес определяется как произведение $\text{tf}(t,d) * \text{idf}(t,d)$

### Сравнение

|Аспект |Bag-of-Words |TF-IDF|
|---|---|---|
|Упорядоченность слов|	игнорируется|	тоже игнорируется|
|Вес слов|	чистый счёт|	счёт * редкость по корпусу|
|Общие слова (stop-words)|	сильно влияют|	приглушаются (низкий IDF)|
|Зависимость от длины|	сильно зависит|	менее чувствителен к длине документов|
|Интерпретируемость|	простая|	чуть сложнее, но информативнее|

## Пример использования векторизации

In [17]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [18]:
texts = [
    "The dog barks at night",
    "The dog sleeps all day",
    "Night owl sees the dog"
]

In [19]:
#BoW
cv = CountVectorizer(stop_words='english')
X_bow = cv.fit_transform(texts)

In [23]:
print("BoW feature names:", cv.get_feature_names_out())
print("Bow vectors:\n", X_bow.toarray())

BoW feature names: ['barks' 'day' 'dog' 'night' 'owl' 'sees' 'sleeps']
Bow vectors:
 [[1 0 1 1 0 0 0]
 [0 1 1 0 0 0 1]
 [0 0 1 1 1 1 0]]


In [21]:
#tf-idf
tv = TfidfVectorizer(stop_words='english')
X_tfidf = tv.fit_transform(texts)

In [22]:
print("TF-IDF feature names:", tv.get_feature_names_out())
print("TF-IDF vectors:\n", X_tfidf.toarray())

TF-IDF feature names: ['barks' 'day' 'dog' 'night' 'owl' 'sees' 'sleeps']
TF-IDF vectors:
 [[0.72033345 0.         0.42544054 0.54783215 0.         0.
  0.        ]
 [0.         0.65249088 0.38537163 0.         0.         0.
  0.65249088]
 [0.         0.         0.34520502 0.44451431 0.5844829  0.5844829
  0.        ]]


## Пример обучения логистической регрессии

In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

In [25]:
texts = [
    "This movie was fantastic!",
    "Terrible film. Worst ever",
    "I loved the cinematography", 
    "Bad script, poor acting"
]
y = [1, 0, 1, 0]  # 1 = positive, 0 = negative sentiment

In [26]:
vect = TfidfVectorizer(stop_words='english')
X = vect.fit_transform(texts)

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=41)

In [31]:
model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [36]:
print(classification_report(y_test, y_pred, zero_division=0))
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.50      1.00      0.67         1
           1       0.00      0.00      0.00         1

    accuracy                           0.50         2
   macro avg       0.25      0.50      0.33         2
weighted avg       0.25      0.50      0.33         2

[[1 0]
 [1 0]]


## Зачем нужен Pipeline и GridSearchCV?

**Pipeline** — это последовательность преобразований («трансформеров»), где последний шаг — классификатор (LogisticRegression).

Он гарантирует:
* Трансформеры (напр. TfidfVectorizer) применяются только на тренировочных данных во время кросс-валидации, а на валидационных — только transform() (не fit), чтобы избежать утечки данных.
* Упрощает код: все шаги в один объект, удобно использовать с grid search.
* Параметры каждого шага можно менять через step_name__param_name

**GridSearchCV** перебирает все возможные комбинации параметров и выбирает лучшую модель, затем получает итоговый обученный пайплайн для предсказаний.

* При каждой комбинации GridSearchCV создаёт новый пайплайн, который заново fit TF‑IDF на тренировочных фолдах, а transform на валидационных. Это предотвращает утечку информации из тестовой части.
* Параметры трансформации текста (например ngram_range, max_df, max_features) и параметры классификатора (например C, penalty) оптимизируются вместе.

## Пример кода

In [47]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, StratifiedKFold, train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

In [48]:
texts = [
    "I absolutely loved this product, it works great and exceeded expectations",
    "Terrible experience, the item broke after one use and customer service was unhelpful",
    "Fantastic performance, would recommend to everyone and buy again",
    "Worst purchase ever, completely disappointed and will never return",
    "This is amazing, quality is top notch and I am very satisfied",
    "Poor quality, not worth the money and dysfunctional features",
    "Incredible value and superb build, highly recommended",
    "Do not buy it, it's a waste of money and time",
    "Excellent features, works flawlessly and I'm impressed",
    "Awful product, it stopped working within a day"
]
y = [1, 0, 1, 0, 1, 0, 1, 0, 1, 0]  # 1 = positive sentiment / интересный, 0 = negative / неинтересный

In [53]:
X_train, X_test, y_train, y_test = train_test_split(
    texts, y, test_size=0.2, random_state=42, stratify=y
)

In [54]:
pipe = Pipeline([('tfidf', TfidfVectorizer(stop_words='english')), ('clf', LogisticRegression(class_weight='balanced', max_iter=1000))])

In [55]:
param_grid = {
    'tfidf__ngram_range': [(1,1),(1,2)],
    'tfidf__max_df': [0.75, 1.0],
    'clf__C': [0.1, 1.0, 10.0]
}

In [57]:
grid = GridSearchCV(pipe, param_grid, cv=StratifiedKFold(n_splits=3, shuffle=True, random_state=42), scoring='f1', n_jobs=-1, verbose=1)
grid.fit(X_train, y_train)

Fitting 3 folds for each of 12 candidates, totalling 36 fits


,estimator,Pipeline(step..._iter=1000))])
,param_grid,"{'clf__C': [0.1, 1.0, ...], 'tfidf__max_df': [0.75, 1.0], 'tfidf__ngram_range': [(1, ...), (1, ...)]}"
,scoring,'f1'
,n_jobs,-1
,refit,True
,cv,StratifiedKFo... shuffle=True)
,verbose,1
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,input,'content'


In [61]:
y_pred = grid.predict(X_test)

In [62]:
print("Best params:", grid.best_params_)
print("Best f1:", grid.best_score_)

Best params: {'clf__C': 0.1, 'tfidf__max_df': 0.75, 'tfidf__ngram_range': (1, 1)}
Best f1: 0.16666666666666666


In [63]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       1.0
           1       0.00      0.00      0.00       1.0

    accuracy                           0.00       2.0
   macro avg       0.00      0.00      0.00       2.0
weighted avg       0.00      0.00      0.00       2.0

